In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

spark = SparkSession.builder \
    .appName("CSV_to_Parquet") \
    .getOrCreate()


In [3]:
olist_customers = spark.read.option("header", True).csv("olist_customers_dataset.csv")
olist_orders = spark.read.option("header", True).csv("olist_orders_dataset.csv")
olist_order_reviews = spark.read.option("header", True).csv("olist_order_reviews_dataset.csv")
olist_order_items = spark.read.option("header", True).csv("olist_order_items_dataset.csv")
olist_sellers = spark.read.option("header", True).csv("olist_sellers_dataset.csv")
olist_products = spark.read.option("header", True).csv("olist_products_dataset.csv")

In [12]:
"""olist_customers.write.mode("overwrite").parquet("olist_customers_dataset.parquet")
olist_orders.write.mode("overwrite").parquet("olist_orders_dataset.parquet")
olist_order_reviews.write.mode("overwrite").parquet("olist_order_reviews_dataset.parquet")
olist_order_items.write.mode("overwrite").parquet("olist_order_items_dataset.parquet")
olist_sellers.write.mode("overwrite").parquet("olist_sellers_dataset.parquet")
olist_products.write.mode("overwrite").parquet("olist_products_dataset.parquet")"""

'olist_customers.write.mode("overwrite").parquet("olist_customers_dataset.parquet")\nolist_orders.write.mode("overwrite").parquet("olist_orders_dataset.parquet")\nolist_order_reviews.write.mode("overwrite").parquet("olist_order_reviews_dataset.parquet")\nolist_order_items.write.mode("overwrite").parquet("olist_order_items_dataset.parquet")\nolist_sellers.write.mode("overwrite").parquet("olist_sellers_dataset.parquet")\nolist_products.write.mode("overwrite").parquet("olist_products_dataset.parquet")'

In [13]:
datasets = [olist_customers, olist_orders, olist_order_reviews, 
            olist_order_items, olist_sellers, olist_products]
for df in datasets:
    df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable

In [8]:
olist_products.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [22]:
olist_products.createOrReplaceTempView("products")

query = spark.sql("""
    SELECT 
        product_category_name AS categoria,
        COUNT(DISTINCT product_id) AS produtos
    FROM products
    GROUP BY product_category_name
    SORT BY produtos DESC
""")

query.show()

+--------------------+--------+
|           categoria|produtos|
+--------------------+--------+
|     cama_mesa_banho|    3029|
|       esporte_lazer|    2867|
|    moveis_decoracao|    2657|
|        beleza_saude|    2444|
|utilidades_domest...|    2335|
|          automotivo|    1900|
|informatica_acess...|    1639|
|          brinquedos|    1411|
|  relogios_presentes|    1329|
|           telefonia|    1134|
|               bebes|     919|
|          perfumaria|     868|
|fashion_bolsas_e_...|     849|
|           papelaria|     849|
|          cool_stuff|     789|
|  ferramentas_jardim|     753|
|            pet_shop|     719|
|                NULL|     610|
|         eletronicos|     517|
|construcao_ferram...|     400|
+--------------------+--------+
only showing top 20 rows



In [16]:
olist_products.createOrReplaceTempView("products")

query = spark.sql("""
    SELECT COUNT(DISTINCT product_id) AS produtos,
           COUNT(DISTINCT product_category_name) AS categoria
    FROM products
""")

query.show()

+--------+---------+
|produtos|categoria|
+--------+---------+
|   32951|       73|
+--------+---------+



In [2]:
# Inicializando a SparkSession
spark = SparkSession.builder.appName("Top3Salaries").getOrCreate()

# Criando o DataFrame de exemplo
data = [
    ("Alice", "TI", 5000),
    ("Bob", "TI", 7000),
    ("Carol", "TI", 6500),
    ("Dave", "RH", 4500),
    ("Eve", "RH", 6000),
    ("Frank", "RH", 6200),
    ("Grace", "RH", 6100)
]
df = spark.createDataFrame(data, ["employee_name", "department", "salary"])

# Definindo a janela particionada por departamento
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

# Criando a coluna de ranking
df = df.withColumn("rank", row_number().over(window_spec))

# Filtrando os top 3 por departamento
df.filter(col("rank") <= 3).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Frank|        RH|  6200|   1|
|        Grace|        RH|  6100|   2|
|          Eve|        RH|  6000|   3|
|          Bob|        TI|  7000|   1|
|        Carol|        TI|  6500|   2|
|        Alice|        TI|  5000|   3|
+-------------+----------+------+----+

